# Breast Cancer Classification using Logistic Regression and ANN

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
# Load the dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/breast-cancer-wisconsin.data"
column_names = ['id', 'clump_thickness', 'uniformity_of_cell_size', 
                'uniformity_of_cell_shape', 'marginal_adhesion', 
                'single_epithelial_cell_size', 'bare_nuclei', 
                'bland_chromatin', 'normal_nucleoli', 
                'mitoses', 'class']
data = pd.read_csv(url, names=column_names)

In [3]:
data.shape

(699, 11)

In [4]:
data.head()

,id,clump_thickness,uniformity_of_cell_size,uniformity_of_cell_shape,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [5]:
# Drop the 'id' column as it's not useful for classification
data.drop('id', axis=1, inplace=True)

In [6]:
data.sample(10)

,clump_thickness,uniformity_of_cell_size,uniformity_of_cell_shape,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
126,7,5,3,7,4,10,7,5,5,4
435,10,8,10,1,3,10,5,1,1,4
412,10,10,10,6,8,4,8,5,1,4
632,1,1,1,1,2,1,1,1,1,2
419,2,3,1,1,5,1,1,1,1,2
399,1,2,3,1,2,1,1,1,1,2
363,4,4,4,2,2,3,2,1,1,2
96,2,1,1,2,2,1,1,1,1,2
297,5,4,3,1,2,?,2,3,1,2
215,8,7,8,7,5,5,5,10,2,4


In [7]:
# Replace '?' with NaN and drop rows with NaN
data.replace('?', np.nan, inplace=True)
data.dropna(inplace=True)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 683 entries, 0 to 698
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   clump_thickness              683 non-null    int64 
 1   uniformity_of_cell_size      683 non-null    int64 
 2   uniformity_of_cell_shape     683 non-null    int64 
 3   marginal_adhesion            683 non-null    int64 
 4   single_epithelial_cell_size  683 non-null    int64 
 5   bare_nuclei                  683 non-null    object
 6   bland_chromatin              683 non-null    int64 
 7   normal_nucleoli              683 non-null    int64 
 8   mitoses                      683 non-null    int64 
 9   class                        683 non-null    int64 
dtypes: int64(9), object(1)
memory usage: 58.7+ KB


In [9]:
# Convert 'bare_nuclei' to int type
data['bare_nuclei'] = data['bare_nuclei'].astype(int)

In [10]:
data['class'].unique()

array([2, 4], dtype=int64)

In [11]:
# Split the dataset into features and target
X = data.drop('class', axis=1)
y = data['class'].replace({2: 0, 4: 1})  # Convert to binary classification (0: benign, 1: malignant)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
# Logistic Regression Model
log_model = LogisticRegression()
log_model.fit(X_train, y_train)

# Predictions
y_pred = log_model.predict(X_test)

# Evaluation
print("Logistic Regression Model")
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Logistic Regression Model
Accuracy: 0.9562043795620438
              precision    recall  f1-score   support

           0       0.94      0.99      0.96        79
           1       0.98      0.91      0.95        58

    accuracy                           0.96       137
   macro avg       0.96      0.95      0.95       137
weighted avg       0.96      0.96      0.96       137



In [14]:
# ANN

# Define the model
nn_model = Sequential()
nn_model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
nn_model.add(Dense(32, activation='relu'))
nn_model.add(Dense(1, activation='sigmoid'))  # Binary classification

In [15]:
# Compile the model
nn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
nn_model.fit(X_train, y_train, epochs=100, batch_size=16, validation_split=0.2)

Epoch 1/100
28/28 [==============================] - 2s 17ms/step - loss: 0.5786 - accuracy: 0.8211 - val_loss: 0.3894 - val_accuracy: 0.9545
Epoch 2/100
28/28 [==============================] - 0s 5ms/step - loss: 0.2953 - accuracy: 0.9679 - val_loss: 0.1841 - val_accuracy: 0.9545
Epoch 3/100
28/28 [==============================] - 0s 5ms/step - loss: 0.1375 - accuracy: 0.9702 - val_loss: 0.1239 - val_accuracy: 0.9545
Epoch 4/100
28/28 [==============================] - 0s 5ms/step - loss: 0.0882 - accuracy: 0.9725 - val_loss: 0.1206 - val_accuracy: 0.9545
Epoch 5/100
28/28 [==============================] - 0s 4ms/step - loss: 0.0731 - accuracy: 0.9771 - val_loss: 0.1234 - val_accuracy: 0.9545
Epoch 6/100
28/28 [==============================] - 0s 4ms/step - loss: 0.0666 - accuracy: 0.9771 - val_loss: 0.1281 - val_accuracy: 0.9545
Epoch 7/100
28/28 [==============================] - 0s 4ms/step - loss: 0.0628 - accuracy: 0.9794 - val_loss: 0.1340 - val_accuracy: 0.9545
Epoch 8/100


Epoch 59/100
28/28 [==============================] - 0s 4ms/step - loss: 0.0162 - accuracy: 0.9977 - val_loss: 0.2467 - val_accuracy: 0.9364
Epoch 60/100
28/28 [==============================] - 0s 4ms/step - loss: 0.0124 - accuracy: 0.9954 - val_loss: 0.2431 - val_accuracy: 0.9545
Epoch 61/100
28/28 [==============================] - 0s 4ms/step - loss: 0.0124 - accuracy: 0.9954 - val_loss: 0.2510 - val_accuracy: 0.9545
Epoch 62/100
28/28 [==============================] - 0s 3ms/step - loss: 0.0107 - accuracy: 0.9954 - val_loss: 0.2500 - val_accuracy: 0.9545
Epoch 63/100
28/28 [==============================] - 0s 4ms/step - loss: 0.0102 - accuracy: 0.9977 - val_loss: 0.2543 - val_accuracy: 0.9545
Epoch 64/100
28/28 [==============================] - 0s 4ms/step - loss: 0.0095 - accuracy: 0.9977 - val_loss: 0.2596 - val_accuracy: 0.9545
Epoch 65/100
28/28 [==============================] - 0s 4ms/step - loss: 0.0087 - accuracy: 0.9977 - val_loss: 0.2576 - val_accuracy: 0.9545
Epoch 

In [16]:
# Evaluate the model
loss, accuracy = nn_model.evaluate(X_test, y_test)
print(f"\nNeural Network Model Accuracy: {accuracy:.4f}")

5/5 [==============================] - 0s 2ms/step - loss: 0.0983 - accuracy: 0.9635

Neural Network Model Accuracy: 0.9635
